### Empezamos importando las librerias que vamos a usar 
En esta parte del ETL ya que, senti que necesitaba obtener todos los datos de los años, a demás tambien queria tener todas las criticas que obtuvieron los juegos hice dos pequeños script para poder extraer esos datos de las diferentes url, por eso importe bs4 y requests

In [3]:
import pandas as pd 
from bs4 import BeautifulSoup 
import requests
import ast

In [4]:
rows  = []
with open('../data/json/steam_games.json') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df = pd.DataFrame(rows)



Hacemos una copia asi si modificamos no perdemos todos los datos

In [5]:
df_copy_stream = df.copy()

<h4>Todo lo que este en descuento como NAN lo llenamos de 0</h4>

In [6]:
df_copy_stream["discount_price"].fillna(0, inplace=True)

In [7]:
df_copy_stream['release_date'].unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2016-11-19',
       'January 2018', '2018-10-01'], dtype=object)

<h4>Por que transformo release_date a una columna año con string?, porque necesito separar los diferentes datos que son mes y dia siento que son datos inutiles para poder solo usar el año de lanzamiento </h4>

In [8]:
df_copy_stream['año'] = pd.to_datetime(df_copy_stream['release_date'], errors='coerce', format='%Y-%m-%d').dt.year.astype("string")

In [9]:

df_copy_stream["url"] = df_copy_stream["url"].astype("string")
df_copy_stream["url"] = df_copy_stream["url"].str.strip()


# Todo lo de abajo esta hecho para que no haya datos faltantes en los años 

In [10]:
filas_con_nulos = df[df["release_date"].isnull()]
lista_nan =  filas_con_nulos["url"]

''' esto era de prueba 
url = "http://store.steampowered.com/app/773570/Log"

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
'''

for url in lista_nan:
    try:  
        response = requests.get(url)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        valor = str(soup.find('div', {'class' : 'grid_content grid_date'}).text.strip().split(",")[1].strip())
        df_copy_stream.loc[df_copy_stream['url'].str.contains(url), 'año'] = valor

    except Exception as e : 
        print(f"Error en URL: {url}. {e}")
        


Error en URL: http://store.steampowered.com/app/662570/United_Tactics/. list index out of range
Error en URL: http://store.steampowered.com/app/769350/Medicalholodeck_Personal_Version/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/202170/Sleeping_Dogs/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/223850/3DMark/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/469560/Hyper_Bowling_VR/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/456570/MuvLuv_VR/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/516830/Grapply/. list index out of range
Error en URL: http://store.steampowered.com/app/426610/Virtually_Live_presents_Formula_E_Season_Two_Highlights/. 'NoneType' object has no attribute 'te

In [24]:
df_copy_stream["año"].isnull().sum()

350

## Este script tambien esta para llenar los datos faltantes de sentiment usando tambien la libreria de BS4 para hacer scrapping en steam pase de ("7128 nulls a 4000")

In [12]:
## Este codigo es para transformar los sentimientos que eran un object a string para poder guardar las diferentes criticas sobre el juego que no estaban en el dataset
df_copy_stream["sentiment"]  = df_copy_stream["sentiment"].astype("string")

In [25]:
## Verificiamos el nulo
df_copy_stream["sentiment"].isnull().sum()

4300

In [14]:
filas_con_nulos = df_copy_stream[df_copy_stream["sentiment"].isnull()]
lista_nan =  filas_con_nulos["url"]



for url in lista_nan:
        try:  
                response = requests.get(url)

                soup = BeautifulSoup(response.content, 'html.parser')
                
                valor = soup.find('span', {'class' : 'game_review_summary'}).text
                df_copy_stream.loc[df_copy_stream["url"].str.contains(url), "sentiment"] = valor
        except Exception as e : 
                print(f"Error en URL: {url}. {e}")
                

Error en URL: http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/774276/SNOW__All_Access_Basic_Pass/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/774277/SNOW__All_Access_Pro_Pass/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/774278/SNOW__All_Access_Legend_Pass/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/774271/SNOW__Starter_Pack/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/774279/SNOW__Lifetime_Pack/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/772590/After_Life_VR/. 'NoneType' object has no attribute 'text'
Error en URL: http://store.steampowered.com/app/775880/RC_Plane_3__Canyon_Scenario/. 'NoneType' object has no attribute 'text'
Error en URL: http://sto

# Solo quedaron esta cantidad de juegos con NaN ya que decian proximamentes o no tenian fecha de lanzamiento

### Acá usamos la función pd.to_numeric para pasar mi columna año que era string a float ya uqe me tiraba un error por tener NaN, lo hice para despues poder usarlo

In [22]:
df_copy_stream['año_v'] = pd.to_numeric(df_copy_stream['año'], errors='coerce').astype('float64')


In [27]:
df_copy_stream.drop(columns="año",inplace=True)

In [28]:
df_copy_stream.drop(columns="release_date", inplace=True)

In [29]:
df_copy_stream['año_v'].isnull().sum()

350

In [30]:
df_copy_stream["sentiment"].isnull().sum()

4300

In [31]:

df[df["id"].isna()]

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)",Overwhelmingly Positive,91


In [20]:
idNanError =  df[df["url"] == ("http://store.steampowered.com/")]
df.drop(idNanError.index, inplace=True)

Acá ya guarde el archivo con su respectivos cambios en la primera parte

In [21]:
s = df_copy_stream.to_csv("../data/csv/steam_games_v1.csv",index=False)